In [14]:
import numpy as np
import pandas as pd

In [2]:
output = []
for region_id in range(1, 15 + 1):
    path = f"https://www.servel.cl/wp-content/uploads/2023/06/{str(region_id).zfill(2)}_Resultados_Mesa_Concejales_Ter.xlsx"
    frag = pd.read_excel(path)
    output.append(frag)

df = pd.concat(output, ignore_index=True)
df.head()

,Región,Provincia,Circ. Senatorial,Distrito,Comuna,Circ. Electoral,Nro. Mesa,Tipo Mesa,Nro. En Voto,Pacto,Sub-Pacto,Partido,Candidato,Votos TER
0,1.0,IQUIQUE,1.0,2.0,IQUIQUE,IQUIQUE,1.0,M,7.0,IGUALDAD PARA CHILE,SIN SUB PACTO,PARTIDO IGUALDAD,JORGE SILVA AMESTICA,1.0
1,1.0,IQUIQUE,1.0,2.0,IQUIQUE,IQUIQUE,1.0,M,8.0,IGUALDAD PARA CHILE,SIN SUB PACTO,INDEPENDIENTES LISTA A,OCTAVIO CANCINO GATICA,0.0
2,1.0,IQUIQUE,1.0,2.0,IQUIQUE,IQUIQUE,1.0,M,9.0,REGIONALISTAS E INDEPENDIENTES,SIN SUB PACTO,PARTIDO REGIONALISTA DE LOS INDEPENDIENTES,GLADYS PEREZ DIAZ,0.0
3,1.0,IQUIQUE,1.0,2.0,IQUIQUE,IQUIQUE,1.0,M,10.0,REGIONALISTAS E INDEPENDIENTES,SIN SUB PACTO,PARTIDO REGIONALISTA DE LOS INDEPENDIENTES,MIGUEL ANGEL OLIVARES MUÑOZ,0.0
4,1.0,IQUIQUE,1.0,2.0,IQUIQUE,IQUIQUE,1.0,M,11.0,REGIONALISTAS E INDEPENDIENTES,SIN SUB PACTO,PARTIDO REGIONALISTA DE LOS INDEPENDIENTES,JESUS VICTOR MESIAS SOTO,1.0


In [20]:
def create_polling_id(data, columns=[]):
    output = data[columns[0]].astype(str).fillna("###").copy()

    for column in columns[1:]:
        output += "-" + data[column].fillna("###").astype(str)
    return output

df["polling_id"] = create_polling_id(df, columns=["Región", "Provincia", "Circ. Senatorial", "Distrito", "Comuna", "Circ. Electoral", "Nro. Mesa", "Tipo Mesa"])

In [21]:
# VOTOS NULOS
# VOTOS EN BLANCO	
# TOTALES
df["Candidato"] = df["Candidato"].replace({
    "VOTOS EN BLANCO": "BLANK",
    "VOTOS BLANCOS": "BLANK",
    "VOTOS NULOS": "NULL"
})

df_spoilt = df[df["Candidato"].isin(["NULL", "BLANK"])].copy()
df_spoilt = df_spoilt[["Candidato", "Votos TER", "polling_id"]].rename(columns={"Candidato": "candidate", "Votos TER": "value"})
df_spoilt["flag_candidates"] = 0
df_spoilt.head()

df = df[~df["Candidato"].isin(["BLANK", "NULL", "TOTALES", np.nan])].copy()


C:\Users\cnava\AppData\Local\Temp\ipykernel_22088\1336623883.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["flag_candidates"] = 1


In [24]:
data = df[["polling_id", "Pacto", "Votos TER"]].dropna(subset=["Pacto"])
data = data.rename(columns={"Pacto": "candidate", "Votos TER": "value"})

tt = data.groupby(["polling_id", "candidate"]).agg({"value": "sum"})
tt["rate"] = tt.groupby(level=[0], group_keys=False).apply(lambda x: x/x.sum()).fillna(0)
tt = tt.reset_index()
tt["flag_candidates"] = 1
tt.head()

,polling_id,candidate,value,rate,flag_candidates
0,1.0-IQUIQUE-1.0-2.0-ALTO HOSPICIO-ALTO HOSPICI...,COALICION,30.0,0.181818,1
1,1.0-IQUIQUE-1.0-2.0-ALTO HOSPICIO-ALTO HOSPICI...,CONCERTACION DEMOCRATICA,33.0,0.200000,1
2,1.0-IQUIQUE-1.0-2.0-ALTO HOSPICIO-ALTO HOSPICI...,EL CAMBIO POR TI,2.0,0.012121,1
3,1.0-IQUIQUE-1.0-2.0-ALTO HOSPICIO-ALTO HOSPICI...,MAS HUMANOS,2.0,0.012121,1
4,1.0-IQUIQUE-1.0-2.0-ALTO HOSPICIO-ALTO HOSPICI...,POR EL DESARROLLO DEL NORTE,58.0,0.351515,1


In [25]:
data = pd.concat([
    tt,
    df_spoilt
])

In [27]:
df_location = df[["polling_id", "Región", "Provincia", "Circ. Senatorial", "Distrito", "Comuna", "Circ. Electoral", "Nro. Mesa", "Tipo Mesa"]].drop_duplicates()
df_location = df_location.rename(columns={
    "Región": "region",
    "Comuna": "commune",
    "Provincia": "province",
    "Distrito": "district",
    "Circ. Senatorial": "senate",
    "Circ. Electoral": "circunscription",
    "Nro. Mesa": "polling_station",
    "Tipo Mesa": "type"
})
# "Región", "Provincia", "Circ. Senatorial", "Distrito", "Comuna", "Circ. Electoral", "Nro. Mesa", "Tipo Mesa"

In [28]:
data.to_csv("../data_output/Chile/2012_councilor.csv.gz")
df_location.to_csv("../data_output/Chile/2012_councilor_location.csv.gz")